In [1]:
exp_name = "v01"

In [2]:
collections_to_areas = {
    '2020-05-14-US-MTV-1': 'highway',
    '2020-05-14-US-MTV-2': 'highway',
    '2020-05-21-US-MTV-1': 'highway',
    '2020-05-21-US-MTV-2': 'highway',
    '2020-05-29-US-MTV-1': 'highway',
    '2020-05-29-US-MTV-2': 'highway',
    '2020-06-04-US-MTV-1': 'highway',
    '2020-06-05-US-MTV-1': 'highway',
    '2020-06-05-US-MTV-2': 'highway',
    '2020-06-11-US-MTV-1': 'highway',
    '2020-07-08-US-MTV-1': 'highway',
    '2020-07-17-US-MTV-1': 'highway',
    '2020-07-17-US-MTV-2': 'highway',
    '2020-08-03-US-MTV-1': 'highway',
    '2020-08-06-US-MTV-2': 'highway',
    '2020-09-04-US-SF-1': 'highway',
    '2020-09-04-US-SF-2': 'highway',
    '2021-01-04-US-RWC-1': 'highway',
    '2021-01-04-US-RWC-2': 'highway',
    '2021-01-05-US-SVL-1': 'highway',
    '2021-01-05-US-SVL-2': 'highway',
    '2021-03-10-US-SVL-1': 'tree',
    '2021-04-15-US-MTV-1': 'tree',
    '2021-04-22-US-SJC-1': 'downtown',
    '2021-04-26-US-SVL-1': 'tree',
    '2021-04-28-US-MTV-1': 'tree',
    '2021-04-28-US-SJC-1': 'downtown',
    '2021-04-29-US-MTV-1': 'tree',
    '2021-04-29-US-SJC-2': 'downtown',
'2020-05-15-US-MTV-1':'highway',
 '2020-05-28-US-MTV-1':'highway',
 '2020-05-28-US-MTV-2':'highway',
 '2020-06-04-US-MTV-2':'highway',
 '2020-06-10-US-MTV-1':'highway',
 '2020-06-10-US-MTV-2':'highway',
 '2020-08-03-US-MTV-2':'highway',
 '2020-08-13-US-MTV-1':'highway',
 '2021-03-16-US-MTV-2':'highway',
 '2021-03-16-US-RWC-2':'tree',
 '2021-03-25-US-PAO-1':'tree',
 '2021-04-02-US-SJC-1':'tree',
 '2021-04-08-US-MTV-1':'tree',
 '2021-04-21-US-MTV-1':'tree',
 '2021-04-22-US-SJC-2': 'downtown',
 '2021-04-26-US-SVL-2':'tree',
 '2021-04-28-US-MTV-2':'tree',
 '2021-04-29-US-MTV-2':'tree',
 '2021-04-29-US-SJC-3': 'downtown',
}

In [3]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm import tqdm
import pathlib
import plotly
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt 
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
import lightgbm as lgb
from sklearn.model_selection import KFold,GroupKFold
from sklearn.metrics import mean_squared_error

# utils

In [4]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [5]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)
    
def visualize_collection_with_gt(df,ground_truth,collection):
    tmp1 = df.copy()
    tmp1['phoneName'] = tmp1['phoneName'] + '_BASE'
    tmp2 = ground_truth.copy()
    tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
    tmp = pd.concat([tmp1, tmp2])    
    visualize_collection(tmp,collection)

# data prep

In [6]:
# directory setting
INPUT = '../input/google-smartphone-decimeter-challenge/'

In [7]:
base_train = pd.read_csv(INPUT + 'baseline_locations_train.csv')
base_test = pd.read_csv(INPUT +  'baseline_locations_test.csv')
sample_sub = pd.read_csv(INPUT + 'sample_submission.csv')

In [8]:
# ground_truth
p = pathlib.Path(INPUT)
gt_files = list(p.glob('train/*/*/ground_truth.csv'))
gts = []
for gt_file in tqdm(gt_files):
    gts.append(pd.read_csv(gt_file))
ground_truth = pd.concat(gts)

100%|██████████| 73/73 [00:00<00:00, 226.69it/s]


In [9]:
ground_truth["area"] = ground_truth["collectionName"].map(collections_to_areas)

# reject outlier

In [10]:
def add_distance_diff(df):
    df['latDeg_prev'] = df['latDeg'].shift(1)
    df['latDeg_next'] = df['latDeg'].shift(-1)
    df['lngDeg_prev'] = df['lngDeg'].shift(1)
    df['lngDeg_next'] = df['lngDeg'].shift(-1)
    df['phone_prev'] = df['phone'].shift(1)
    df['phone_next'] = df['phone'].shift(-1)
    
    df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
    df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
    
    df.loc[df['phone']!=df['phone_prev'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
    df.loc[df['phone']!=df['phone_next'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
    
    return df

# kalman filter
https://www.kaggle.com/emaerthin/demonstration-of-the-kalman-filter

In [11]:
#!pip install simdkalman

In [12]:
import simdkalman

In [13]:
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in unique_paths:
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

# evaluate train score

In [14]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [15]:
def get_train_score(df, gt,area=False):
    cols_for_score = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg','lngDeg']
    df = df[cols_for_score]
    gt = gt[cols_for_score]
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby(["collectionName",'phone'])['err'].agg([percentile50, percentile95]).reset_index()
    res["area"] = res["collectionName"].map(collections_to_areas)
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    if area:
        print(res.groupby(["area"])['p50_p90_mean'].mean())
    score = res['p50_p90_mean'].mean()
    return score

### 1. make KF baseline

In [16]:
# reject outlier
train_ro = add_distance_diff(base_train)
th = 50
train_ro.loc[((train_ro['dist_prev'] > th) & (train_ro['dist_next'] > th)), ['latDeg', 'lngDeg']] = np.nan
cols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']
train_pred = apply_kf_smoothing(train_ro[cols])
train_pred.to_pickle("train_kf.pkl")

/root/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [17]:
test_ro = add_distance_diff(base_test)
th = 50
test_ro.loc[((test_ro['dist_prev'] > th) & (test_ro['dist_next'] > th)), ['latDeg', 'lngDeg']] = np.nan
cols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']
test_pred = apply_kf_smoothing(test_ro[cols])
test_pred.to_pickle("test_kf.pkl")

/root/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### 2. mean pred and denoise stopping


In [18]:
def make_lerp_data(df):
    '''
    Generate interpolated lat,lng values for different phone times in the same collection.
    '''
    org_columns = df.columns

    # Generate a combination of time x collection x phone and combine it with the original data (generate records to be interpolated)
    time_list = df[['collectionName', 'millisSinceGpsEpoch']].drop_duplicates()
    phone_list =df[['collectionName', 'phoneName']].drop_duplicates()
    tmp = time_list.merge(phone_list, on='collectionName', how='outer')

    lerp_df = tmp.merge(df, on=['collectionName', 'millisSinceGpsEpoch', 'phoneName'], how='left')
    lerp_df['phone'] = lerp_df['collectionName'] + '_' + lerp_df['phoneName']
    lerp_df = lerp_df.sort_values(['phone', 'millisSinceGpsEpoch'])
    

    lerp_df = lerp_df.set_index("millisSinceGpsEpoch")
    lerp_df["latDeg_"]=lerp_df.groupby('phone')["latDeg"].apply(
        lambda group: group.interpolate(method='index',limit_area='inside')).values
    lerp_df["lngDeg_"]=lerp_df.groupby('phone')["lngDeg"].apply(
        lambda group: group.interpolate(method='index',limit_area='inside')).values
    lerp_df = lerp_df.reset_index()
    lerp_df = lerp_df[(lerp_df["latDeg"].isnull())&(lerp_df["latDeg_"].notnull())]
    lerp_df["latDeg"] = lerp_df["latDeg_"]
    lerp_df["lngDeg"] = lerp_df["lngDeg_"]    
    return lerp_df[org_columns]

In [19]:
def calc_mean_pred(df, lerp_df,weights_list):
    weights_dic ={
        'Pixel4':weights_list[0],
        'Pixel4XLModded':weights_list[1],
        'Pixel4XL':weights_list[2], 
        'Mi8':weights_list[3], 
        'Pixel4Modded':weights_list[4],
        'Pixel5':weights_list[5], 
        'SamsungS20Ultra':weights_list[6],
    }
    '''
    Make a prediction based on the average of the predictions of phones in the same collection.
    '''
    add_lerp = pd.concat([df, lerp_df])
    add_lerp["weights"] = add_lerp["phoneName"].map(weights_dic)

    add_lerp["latDeg_w"] = add_lerp["latDeg"]*add_lerp["weights"]
    add_lerp["lngDeg_w"] = add_lerp["lngDeg"]*add_lerp["weights"]

    mean_pred_result = add_lerp.groupby(['collectionName', 'millisSinceGpsEpoch'])[["latDeg_w","lngDeg_w","weights"]].sum().reset_index()

    mean_pred_result["latDeg"] = mean_pred_result["latDeg_w"]/mean_pred_result["weights"]
    mean_pred_result["lngDeg"] = mean_pred_result["lngDeg_w"]/mean_pred_result["weights"]

    mean_pred_df = df[['collectionName', 'phoneName', 'millisSinceGpsEpoch']].copy()
    mean_pred_df = mean_pred_df.merge(mean_pred_result[['collectionName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['collectionName', 'millisSinceGpsEpoch'], how='left')
    return mean_pred_df

In [20]:
def fill_stopping(df,thresh=2,window=30):

    df["phone"] = df["collectionName"]+"_"+df["phoneName"]
    df["latDeg_shift_1"] = df.groupby(["phone"])["latDeg"].shift()
    df["lngDeg_shift_1"] = df.groupby(["phone"])["lngDeg"].shift()
    df["dist_shift1"] = calc_haversine(
        df['latDeg'],
        df['lngDeg'],
        df["latDeg_shift_1"],
        df["lngDeg_shift_1"]
    )
    idx_stopping = df["dist_shift1"]<thresh
    df["latDeg_rolling_median"] = df.groupby('phone')[
        'latDeg'].transform(lambda s: s.rolling(window=window, center=True,min_periods = 1).median())
    df["lngDeg_rolling_median"] = df.groupby('phone')[
        'lngDeg'].transform(lambda s: s.rolling(window=window, center=True,min_periods = 1).median())
    df.loc[idx_stopping,"latDeg"] = df.loc[idx_stopping,"latDeg_rolling_median"]
    df.loc[idx_stopping,"lngDeg"] = df.loc[idx_stopping,"lngDeg_rolling_median"]    
    return df

In [21]:
def repeat_postprocess(
    df_pred,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    evalate = False,
):
    for i in range(repeats):
        df_lerp = make_lerp_data(df_pred)
        df_pred = calc_mean_pred(df_pred, df_lerp,phone_weights)
        df_pred = fill_stopping(df_pred,thresh=thresh_stopping,window=median_window)
        if evalate:
            score = get_train_score(df_pred, ground_truth)
            print("count : {}, score : {:.4f}".format(i+1,score))
    return df_pred

In [22]:
train_pred = pd.read_pickle("train_kf.pkl")
test_pred = pd.read_pickle("test_kf.pkl")

In [23]:
train_pred["area"] = train_pred["collectionName"].map(collections_to_areas)
test_pred["area"] = test_pred["collectionName"].map(collections_to_areas)

In [24]:
train_pred_all = []
test_pred_all = []

#### 2.1 highway

In [25]:
target_area = 'highway'
thresh_stopping = 2
median_window = 30
repeats = 2
phone_weights = [
    0.8185272369412809,
    0.34843268384881615,
    0.4846644591847249,
    0.9452238465987974,
    0.9986981935560018,
    0.18031216717585496,
    0.445328696935905
    ]

In [26]:
train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = repeat_postprocess(
    train_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
train_pred_all.append(train_pred_area)

In [27]:
test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = repeat_postprocess(
    test_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
test_pred_all.append(test_pred_area)

####  2.2tree

In [28]:
target_area = 'tree'
thresh_stopping = 1.5
median_window = 30
repeats = 7
phone_weights = [
    0.3115193764226983,
    0.196919988137904,
    0.1757293314492492,
    0.4811756152653266,
    0.9662722758231396,
    0.48662174873037445,
    0.1015768847715516
]

In [29]:
train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = repeat_postprocess(
    train_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
train_pred_all.append(train_pred_area)

In [30]:
test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = repeat_postprocess(
    test_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
test_pred_all.append(test_pred_area)

#### 2.3 downtown

In [31]:
target_area = 'downtown'
thresh_stopping = 2
median_window = 17
repeats = 90
phone_weights = [
    0.30701282127989166,
    0.8541717731249667,
    0.1375108003326193,
    0.34089358646969836,
    0.3585169573830474,
    0.253223739710727,
    0.4830857599143877
]

In [32]:
train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = repeat_postprocess(
    train_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
train_pred_all.append(train_pred_area)

In [33]:
test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = repeat_postprocess(
    test_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
test_pred_all.append(test_pred_area)

In [34]:
train_pred = pd.concat(train_pred_all,ignore_index=True)
test_pred = pd.concat(test_pred_all,ignore_index=True)

In [35]:
train_pred.to_pickle(f"train_pp_{exp_name}.pkl")
test_pred.to_pickle(f"test_pp_{exp_name}.pkl")

In [36]:
get_train_score(train_pred, ground_truth,area = True)

area
downtown    13.904846
highway      2.129207
tree         3.361387
Name: p50_p90_mean, dtype: float64


3.333376347713986

### 3. Search GroundTruth Neighour

In [37]:
from scipy import interpolate
def interp(observed):
    rate = 0.1
    x_observed = np.arange(0, len(observed),1)
    x_latent = np.arange(0, len(observed)-1,0.1)
    interp_func = interpolate.interp1d(x_observed,observed)
    latent = interp_func(x_latent)    
    return latent

def snap_to_grid(df,line_points,max_neighbor = 5):
    df = df.copy()
    line_points["phone"] = line_points["collectionName"]+"_"+line_points["phoneName"]
    line_points_interp = pd.DataFrame()
    line_points_interp["latDeg"] = np.concatenate(
        line_points.groupby(["phone"])["latDeg"].apply(
            lambda x:interp(x.values)).values
    )
    line_points_interp["lngDeg"] = np.concatenate(
        line_points.groupby(["phone"])["lngDeg"].apply(
            lambda x:interp(x.values)).values
    )
    line_points = line_points_interp
    line_points = line_points.drop_duplicates(["latDeg","lngDeg"]).reset_index()

    pred_point = df.apply(lambda row:[row["latDeg"],row["lngDeg"]],axis=1).tolist()
    grid_point = line_points.apply(lambda row:[row["latDeg"],row["lngDeg"]],axis=1).tolist()
    neigh = NearestNeighbors(n_neighbors=1,n_jobs = -1)

    neigh.fit(grid_point)
    idx_neighbor_all = neigh.kneighbors(pred_point,max_neighbor , return_distance=False)
    for i in range(max_neighbor):
        idx_neighbor = idx_neighbor_all[:,i]
        df[f"latDeg_nearest_{i}"] = line_points.loc[idx_neighbor]["latDeg"].values
        df[f"lngDeg_nearest_{i}"] = line_points.loc[idx_neighbor]["lngDeg"].values
        df[f"dist_nearest_{i}"] = calc_haversine(df["latDeg"], df["lngDeg"],df[f"latDeg_nearest_{i}"],df[f"lngDeg_nearest_{i}"])
    return df

In [38]:
train_pred = pd.read_pickle(f"train_pp_{exp_name}.pkl")
test_pred = pd.read_pickle(f"test_pp_{exp_name}.pkl")
train_pred["area"] = train_pred["collectionName"].map(collections_to_areas)
test_pred["area"] = test_pred["collectionName"].map(collections_to_areas)

#### 3.1 get nearest GT feature for train

In [39]:
train_snap_all = []
for target_area in train_pred["area"].unique():
    train_pred_area = train_pred[train_pred["area"] == target_area].reset_index(drop=True)
    ground_truth_area = ground_truth[ground_truth["area"] == target_area].reset_index(drop=True)
    target_collections = train_pred_area["collectionName"].unique()
    for target_collection in tqdm(target_collections):
        target_pred_df = train_pred_area[
            train_pred_area["collectionName"]==target_collection].reset_index(drop=True)
        target_gt_df = ground_truth_area[
            (ground_truth_area["collectionName"]!=target_collection)].reset_index(drop=True)
        target_pred_df_snapped = snap_to_grid(target_pred_df,target_gt_df,max_neighbor=5)
        train_snap_all.append(target_pred_df_snapped) 
train_pred = pd.concat(train_snap_all,ignore_index=True)

100%|██████████| 3/3 [00:03<00:00,  1.13s/it]


In [40]:
train_pred.to_pickle(f"train_snap_{exp_name}.pkl")

#### 3.2 get nearest GT feature for test

In [41]:
test_snap_all = []
for target_area in tqdm(test_pred["area"].unique()):
    target_pred_df = test_pred[
        test_pred["area"] == target_area].reset_index(drop=True)
    target_gt_df = ground_truth[
        ground_truth["area"] == target_area].reset_index(drop=True)
    target_pred_df_snapped = snap_to_grid(target_pred_df,target_gt_df,max_neighbor=5)
    test_snap_all.append(target_pred_df_snapped)
test_pred = pd.concat(test_snap_all,ignore_index=True)
test_pred.to_pickle(f"test_snap_{exp_name}.pkl")

100%|██████████| 3/3 [00:19<00:00,  6.55s/it]


### 4. ML

In [42]:
train_pred = pd.read_pickle(f"train_snap_{exp_name}.pkl")
test_pred = pd.read_pickle(f"test_snap_{exp_name}.pkl")

In [43]:
get_train_score(train_pred, ground_truth,area = True)

area
downtown    13.904846
highway      2.129207
tree         3.361387
Name: p50_p90_mean, dtype: float64


3.333376347713986

#### 4.1 add other phone feature

In [45]:
train_lerp = make_lerp_data(train_pred)
train_lerp = train_lerp.pivot(index=['collectionName',  'millisSinceGpsEpoch'], columns='phoneName', values=['latDeg','lngDeg'])
train_lerp.columns = ["other_"+col[1]+"_"+col[0] for col in train_lerp.columns]
train_lerp = train_lerp.reset_index()
train_pred = train_pred.merge(train_lerp,on=['collectionName', 'millisSinceGpsEpoch'],how="left")
test_lerp = make_lerp_data(test_pred)
test_lerp = test_lerp.pivot(index=['collectionName',  'millisSinceGpsEpoch'], columns='phoneName', values=['latDeg','lngDeg'])
test_lerp.columns = ["other_"+col[1]+"_"+col[0] for col in test_lerp.columns]
test_lerp = test_lerp.reset_index()
test_pred = test_pred.merge(test_lerp,on=['collectionName', 'millisSinceGpsEpoch'],how="left")

In [46]:
gt = ground_truth.copy()
gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
train_pred = train_pred.merge(gt[['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg_gt',
       'lngDeg_gt']], on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='left')

In [47]:
def preprocess_for_ml(df):
    df["phone"] = df["collectionName"]+"_"+df["phoneName"]
    df["latDeg_nearest_mean"] = df[[
        col for col in df.columns if "latDeg_nearest" in col]].mean(axis=1)
    df["lngDeg_nearest_mean"] = df[[
        col for col in df.columns if "lngDeg_nearest" in col]].mean(axis=1)
    df["latDeg_diff_p1"] = df.groupby(['phone'])["latDeg"].diff()
    df["lngDeg_diff_p1"] = df.groupby(['phone'])["lngDeg"].diff()
    df["latDeg_diff_m1"] = df.groupby(['phone'])["latDeg"].diff(-1)
    df["lngDeg_diff_m1"] = df.groupby(['phone'])["lngDeg"].diff(-1)
    latdeg_cols = [col for col in df.columns if "latDeg"  in col]
    lngdeg_cols = [col for col in df.columns if "lngDeg"  in col]
    relative_cols = latdeg_cols + lngdeg_cols
    for col in relative_cols:
        if "latDeg" in col:
            df[col+"_rel"] = df[col]-df["latDeg"]
        elif "lngDeg" in col:
            df[col+"_rel"] = df[col]-df["lngDeg"]
    phone_dic = {
        'Pixel4':0,
        'Pixel4XLModded':1,
        'Pixel4XL':2,
        'Mi8':3,
        'Pixel4Modded':4,
        'Pixel5':5,
        'SamsungS20Ultra':6
    }
    df["phoneName_num"] = df["phoneName"].map(phone_dic)
    return df

In [48]:
train_pred = preprocess_for_ml(train_pred)
test_pred = preprocess_for_ml(test_pred)

In [49]:
train_all = []
test_all = []

In [50]:
not_use_cols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'phone', 'latDeg_gt',
 'lngDeg_gt','latDeg_gt_rel','lngDeg_gt_rel',"area"]
use_cols = [col for col in train_pred.columns if col not in not_use_cols]

In [51]:
params = {'boosting': 'gbdt',
          'metric': 'rmse', 
          'objective': 'regression',
          "learning_rate":0.01
         }

### 4.2 run lightgbm

In [52]:
def run_lgb(df_train,df_test,target_col):

    n_fold=3
    folds=GroupKFold(n_splits=n_fold)
    y_train = df_train[target_col]
    oof = np.zeros(len(df_train))
    predictions = np.zeros(len(df_test))
    feature_importance_df = pd.DataFrame()
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train[use_cols],y_train,df_train["collectionName"].values)):
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][use_cols], label=y_train[trn_idx])
        val_data = lgb.Dataset(df_train.iloc[val_idx][use_cols], label=y_train[val_idx])
        num_round = 1500
        model= lgb.train(params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=False,
                        early_stopping_rounds = 200)
        oof[val_idx] = model.predict(df_train.iloc[val_idx][use_cols], num_iteration=model.best_iteration)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = use_cols
        fold_importance_df["importance"] = model.feature_importance("gain")
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        predictions += model.predict(df_test[use_cols], num_iteration=model.best_iteration) / folds.n_splits
    return oof,predictions,feature_importance_df

def show_feature_importance(feature_importance_df):
    cols = (feature_importance_df[["Feature", "importance"]]
            .groupby("Feature")
            .mean()
            .sort_values(by="importance", ascending=False)[:80].index)

    best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

    plt.figure(figsize=(14,15))
    sns.barplot(x="importance",
                y="Feature",
                data=best_features.sort_values(by="importance",
                                               ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()    

####  4.3.1 highway

In [53]:
target_area = "highway"

In [54]:
df_train_area = train_pred[train_pred["area"]==target_area].reset_index(drop=True)
df_test_area = test_pred[test_pred["area"]==target_area].reset_index(drop=True)
oof,predictions,feature_importance_df = run_lgb(
    df_train_area,df_test_area,'latDeg_gt_rel')
df_train_area["latDeg"] = df_train_area["latDeg"] + oof
df_test_area["latDeg"] = df_test_area["latDeg"] +predictions
oof,predictions,feature_importance_df = run_lgb(
    df_train_area,df_test_area,'lngDeg_gt_rel')
df_train_area["lngDeg"] = df_train_area["lngDeg"] +oof
df_test_area["lngDeg"] = df_test_area["lngDeg"] +predictions
train_all.append(df_train_area)
test_all.append(df_test_area)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008540 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18366
[LightGBM] [Info] Number of data points in the train set: 63081, number of used features: 73
[LightGBM] [Info] Start training from score -0.000003
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008450 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18366
[LightGBM] [Info] Number of data points in the train set: 62560, number of used features: 73
[LightGBM] [Info] Start training from score -0.000003
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008585 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18366
[LightGBM] [Info] Number of data points in the train set: 62331, number of used features: 73
[LightGBM] [I

In [55]:
#show_feature_importance(feature_importance_df)

#### 4.3.2 tree

In [56]:
target_area = "tree"

In [57]:
df_train_area = train_pred[train_pred["area"]==target_area].reset_index(drop=True)
df_test_area = test_pred[test_pred["area"]==target_area].reset_index(drop=True)
oof,predictions,feature_importance_df = run_lgb(
    df_train_area,df_test_area,'latDeg_gt_rel')
df_train_area["latDeg"] = df_train_area["latDeg"] + oof
df_test_area["latDeg"] = df_test_area["latDeg"] +predictions
oof,predictions,feature_importance_df = run_lgb(
    df_train_area,df_test_area,'lngDeg_gt_rel')
df_train_area["lngDeg"] = df_train_area["lngDeg"] +oof
df_test_area["lngDeg"] = df_test_area["lngDeg"] +predictions
train_all.append(df_train_area)
test_all.append(df_test_area)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003520 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17342
[LightGBM] [Info] Number of data points in the train set: 15938, number of used features: 69
[LightGBM] [Info] Start training from score -0.000007
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002703 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17324
[LightGBM] [Info] Number of data points in the train set: 14808, number of used features: 69
[LightGBM] [Info] Start training from score -0.000006
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002607 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17328
[LightGBM] [Info] Number of data points in the train set: 14940, number of used features: 69
[LightGBM] [I

#### 4.3.3 downtown

In [58]:
# df_train_area = train_pred[train_pred["area"]==target_area].reset_index(drop=True)
# df_test_area = test_pred[test_pred["area"]==target_area].reset_index(drop=True)
# oof,predictions,feature_importance_df = run_lgb(
#     df_train_area,df_test_area,'latDeg_gt_rel')
# df_train_area["latDeg"] = df_train_area["latDeg"] + oof
# df_test_area["latDeg"] = df_test_area["latDeg"] +predictions
# oof,predictions,feature_importance_df = run_lgb(
#     df_train_area,df_test_area,'lngDeg_gt_rel')
# df_train_area["lngDeg"] = df_train_area["lngDeg"] +oof
# df_test_area["lngDeg"] = df_test_area["lngDeg"] +predictions

In [59]:
def calc_angle(lat1, lon1, lat2, lon2, lat3, lon3):
    """Calculates angle
    """
    vec12 = np.array([lat1, lon1]) - np.array([lat2, lon2]) 
    vec32 = np.array([lat3, lon3]) - np.array([lat2, lon2])
    norm12 = np.linalg.norm(vec12)
    norm32 = np.linalg.norm(vec32)
    cos = np.inner(vec12, vec32) / (norm12 * norm32)
    rad = np.arccos(cos)
    degree = np.rad2deg(rad)
    
    return degree
def speed_predict(train_df, gt_df, test_df):

    use_cols = ["collectionName", "phoneName", "millisSinceGpsEpoch",
                "latDeg", "lngDeg", "phone"]
    if "phone" not in train_df.columns:
        train_df['phone'] = train_df['collectionName'] + '_' + train_df['phoneName']
        test_df['phone'] = test_df['collectionName'] + '_' + test_df['phoneName']
    train_indices = train_df["collectionName"].str.contains("SJC")
    test_indices = test_df["collectionName"].str.contains("SJC")
    trn_df = pd.merge(
        train_df.loc[train_indices, use_cols].reset_index(drop=True),
        gt_df.rename(columns={
            "latDeg": "gt_lat",
            "lngDeg": "gt_lng",
            "heightAboveWgs84EllipsoidM": "gt_heightAboveWgs84EllipsoidM"
        }),
        on=["collectionName", "phoneName", "millisSinceGpsEpoch"],
        how="left"
    )
    tst_df = test_df[test_indices].reset_index(drop=True)

    def time_feature(df):
        df["min_time"] = df.groupby(["collectionName", "phoneName"])["millisSinceGpsEpoch"].transform("min")
        df["max_time"] = df.groupby(["collectionName", "phoneName"])["millisSinceGpsEpoch"].transform("max")
        df["norm_time"] = (df["millisSinceGpsEpoch"] - df["min_time"]) / (df["max_time"] - df["min_time"])
        df.drop(["min_time", "max_time"], axis=1, inplace=True)
        return df
        
    def add_prev_next(df):
        # linear interpolation
        df['latDeg_prev'] = df['latDeg'].shift(1)
        df['latDeg_next'] = df['latDeg'].shift(-1)
        df['lngDeg_prev'] = df['lngDeg'].shift(1)
        df['lngDeg_next'] = df['lngDeg'].shift(-1)
        df['phone_prev'] = df['phone'].shift(1)
        df['phone_next'] = df['phone'].shift(-1)
        df['time_prev'] = df['millisSinceGpsEpoch'].shift(1)
        df['time_next'] = df['millisSinceGpsEpoch'].shift(-1)
        
        df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
        df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
        
        df.loc[df['phone']!=df['phone_prev'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
        df.loc[df['phone']!=df['phone_next'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
        
        df["angle"] = df.apply(lambda x: calc_angle(
            x["latDeg_prev"], x["lngDeg_prev"],
            x["latDeg"], x["lngDeg"],
            x["latDeg_next"], x["lngDeg_next"],
        ), axis=1)
        
        df["speed_prev"] = df["dist_prev"] / (df["millisSinceGpsEpoch"] - df["time_prev"])
        df["speed_next"] = df["dist_next"] / (df["time_next"] - df["millisSinceGpsEpoch"])
        df["speed_change"] = df["speed_next"] - df["speed_prev"]

        return df

    def rolling_feature(df):
        # angle
        calc_cols = ["angle", "dist_next", "speed_next"]
        window_list = [5, 11, 21]
        for col in calc_cols:
            for w in window_list:
                df[f"{col}_{w}_mean"] = df.groupby("phone")[col].rolling(
                    window=w, center=True).mean().reset_index(level=0, drop=True)
                df[f"{col}_{w}_std"] = df.groupby("phone")[col].rolling(
                    window=w, center=True).std().reset_index(level=0, drop=True)
                df[f"{col}_{w}_min"] = df.groupby("phone")[col].rolling(
                    window=w, center=True).min().reset_index(level=0, drop=True)
                df[f"{col}_{w}_max"] = df.groupby("phone")[col].rolling(
                    window=w, center=True).max().reset_index(level=0, drop=True)
        return df
    
    def device_feature(df):
        df["device_same_point_lat_std"] = df.groupby(
            ["collectionName", "millisSinceGpsEpoch"])["latDeg"].transform("std")
        df["device_same_point_lng_std"] = df.groupby(
            ["collectionName", "millisSinceGpsEpoch"])["lngDeg"].transform("std")
        return df

    # train 
    trn_df = time_feature(trn_df)
    trn_df = add_prev_next(trn_df)
    trn_df = rolling_feature(trn_df)
    # test
    tst_df = time_feature(tst_df)
    tst_df = add_prev_next(tst_df)
    tst_df = rolling_feature(tst_df)

    # lightgbm training
    n_folds = 3
    folds = GroupKFold(n_splits=n_folds)

    except_cols = [
        "collectionName",
        "phoneName",
        "millisSinceGpsEpoch",
        "latDeg", "lngDeg",
        "gt_heightAboveWgs84EllipsoidM",
        "phone",
        "gt_lat", "gt_lng",
        "timeSinceFirstFixSeconds",
        "hDop",
        "vDop",
        "speedMps",
        "courseDegree",
        "phone_prev",
        "phone_next",
        "error",
        "area"
    ]

    lgb_params = {
        'objective': 'regression',
        "metric": "rmse",
        'num_leaves': 128,
        'min_data_in_leaf': 30,
        'max_depth': 4,
        'learning_rate': 0.05,
        "boosting": "gbdt",
        "feature_fraction": 0.8,
        "bagging_freq": 1,
        "bagging_fraction": 0.8,
        "lambda_l1": 0.6,
        "lambda_l2": 0.3,
        "seed": 2021,
        "verbosity": -1,
        "num_threads": -1,
    }
    oof_preds = np.zeros(len(trn_df))
    test_preds = np.zeros(len(tst_df))

    fe_df_list = []

    for fold_ix, (trn_idx, val_idx) in enumerate(folds.split(trn_df, groups=trn_df["collectionName"])):
        print(f"Fold {fold_ix}")
        use_feats = [c for c in trn_df.columns if c not in except_cols]
        _trn_df = trn_df.iloc[trn_idx].reset_index(drop=True)
        _val_df = trn_df.iloc[val_idx].reset_index(drop=True)
        
        # make lgb dataset
        trn_data = lgb.Dataset(
            _trn_df[use_feats],
            label=_trn_df["speedMps"]
        )
        val_data = lgb.Dataset(
            _val_df[use_feats],
            label=_val_df["speedMps"]
        )
        model = lgb.train(
            lgb_params,
            trn_data,
            num_boost_round=1000,
            valid_sets=[trn_data, val_data],
            verbose_eval=500,
            early_stopping_rounds=100)
        
        oof_preds[val_idx] = model.predict(_val_df[use_feats])
        test_preds += model.predict(tst_df[use_feats]) / n_folds
        
        fe_df = pd.DataFrame({
            "feature": use_feats,
            "importance": model.feature_importance("gain")
        })
        fe_df_list.append(fe_df)
        
    importance_df = pd.concat(fe_df_list).groupby(
        "feature")["importance"].mean().sort_values(ascending=False).reset_index()
    
    res_df = pd.DataFrame({
        "preds": oof_preds,
        "phone": trn_df["phone"],
        "latDeg": trn_df["latDeg"],
        "lngDeg": trn_df["lngDeg"],
        "gt_lat": trn_df["gt_lat"],
        "gt_lng": trn_df["gt_lng"],
        "speed": trn_df["speedMps"],
    })
    rmse = np.sqrt(mean_squared_error(res_df["speed"], res_df["preds"]))
    print(f"RMSE: {rmse}")

    train_df.loc[train_indices, "speed_pred"] = oof_preds
    test_df.loc[test_indices, "speed_pred"] = test_preds
    
    if 0:
        # if SVC and stop, convert to NaN
        speed_thresh = 2
        time_from = 0.05
        time_to = 0.95

        def time_feature(df):
            df["min_time"] = df.groupby(["collectionName", "phoneName"])["millisSinceGpsEpoch"].transform("min")
            df["max_time"] = df.groupby(["collectionName", "phoneName"])["millisSinceGpsEpoch"].transform("max")
            df["norm_time"] = (df["millisSinceGpsEpoch"] - df["min_time"]) / (df["max_time"] - df["min_time"])
            df.drop(["min_time", "max_time"], axis=1, inplace=True)
            return df

        train_df = time_feature(train_df)
        test_df = time_feature(test_df)

        train_df.loc[
            (train_df["collectionName"].str.contains("SJC"))&
            (train_df["speed_pred"] < speed_thresh)&(
            (train_df["norm_time"] > time_from) & (train_df["norm_time"] < time_to)), "latDeg"] = np.nan
        train_df.loc[
            (train_df["collectionName"].str.contains("SJC"))&
            (train_df["speed_pred"] < speed_thresh)&(
            (train_df["norm_time"] > time_from) & (train_df["norm_time"] < time_to)), "lngDeg"] = np.nan

        test_df.loc[
            (test_df["collectionName"].str.contains("SJC"))&
            (test_df["speed_pred"] < speed_thresh)&(
            (test_df["norm_time"] > time_from) & (test_df["norm_time"] < time_to)), "latDeg"] = np.nan
        test_df.loc[
            (test_df["collectionName"].str.contains("SJC"))&
            (test_df["speed_pred"] < speed_thresh)&(
            (test_df["norm_time"] > time_from) & (test_df["norm_time"] < time_to)), "lngDeg"] = np.nan

        train_df["latDeg"] = train_df.groupby(["collectionName", "phoneName"])["latDeg"].apply(
            lambda group: group.interpolate(method="linear")).fillna(method="bfill").fillna(method="ffill")
        train_df["lngDeg"] = train_df.groupby(["collectionName", "phoneName"])["lngDeg"].apply(
            lambda group: group.interpolate(method="linear")).fillna(method="bfill").fillna(method="ffill")

        test_df["latDeg"] = test_df.groupby(["collectionName", "phoneName"])["latDeg"].apply(
            lambda group: group.interpolate(method="linear")).fillna(method="bfill").fillna(method="ffill")
        test_df["lngDeg"] = test_df.groupby(["collectionName", "phoneName"])["lngDeg"].apply(
            lambda group: group.interpolate(method="linear")).fillna(method="bfill").fillna(method="ffill")

        del train_df["norm_time"], test_df["norm_time"]

    return train_df, test_df

def minimalize_search(query, ref):

    n_query = len(query)

    fit_lat_list = []
    fit_lng_list = []
    fit_cname_list = []
    fit_timediff_list = []
    fit_angle_list = []
    fit_dist_list = []

    # 最初はqueryのまま
    fit_lat_list.append(query["latDeg"].iloc[0])
    fit_lng_list.append(query["lngDeg"].iloc[0])
    fit_cname_list.append(None)
    fit_timediff_list.append(None)
    fit_angle_list.append(None)
    fit_dist_list.append(0)

    # その次以降
    for ix in tqdm(range(1, len(query))):

        query_lat = query["latDeg"].iloc[ix]
        query_lng = query["lngDeg"].iloc[ix]
        query_speed = query["speed_pred"].iloc[ix]

        prev_lat = fit_lat_list[-1]
        prev_lng = fit_lng_list[-1]
        prev_dist = fit_dist_list[-1]
        prev_cname = fit_cname_list[-1]

        # 前の点と同じだった場合
        if query_lat == prev_lat and query_lng == prev_lng:
            fit_lat_list.append(query_lat)
            fit_lng_list.append(query_lng)
            fit_cname_list.append(fit_cname_list[-1])
            fit_timediff_list.append(fit_timediff_list[-1])
            fit_angle_list.append(None)
            fit_dist_list.append(0)


        else:
            dists = calc_haversine(query_lat, query_lng, ref["latDeg"].values, ref["lngDeg"].values)

            if dists.min() > 100:
                fit_lat_list.append(query_lat)
                fit_lng_list.append(query_lng)
                fit_cname_list.append(fit_cname_list[-1])
                fit_timediff_list.append(fit_timediff_list[-1])
                fit_angle_list.append(None)
                continue


            candidate0_lat = query_lat
            candidate0_lng = query_lng

            argsort_indices = np.argsort(dists)

            candidates = ref.iloc[argsort_indices[:30]].reset_index(drop=True)
            candidates_lat = candidates["latDeg"]
            candidates_lng = candidates["lngDeg"]
            candidates_cname = candidates["collectionName"]
            candidates_time = candidates["millisSinceGpsEpoch"]

            # predとの距離
            pred_dists = calc_haversine(
                query_lat, query_lng,
                candidates_lat,
                candidates_lng
            )
            # 前のfitポイントとの距離
            prev_fit_dists = calc_haversine(
                prev_lat, prev_lng,
                candidates_lat,
                candidates_lng
            )
            
            cost_pred_dist = pred_dists.values
            cost_prev_dist = np.abs(prev_fit_dists - query_speed).values
            cost_pred_fit_dist = np.abs(pred_dists - prev_dist).values
            cost_cname = (prev_cname ==candidates_cname).values.astype(float)
            
            cost_total = cost_pred_dist * 0.5 + cost_prev_dist * 0.5+cost_pred_fit_dist*0.3+0.5*(cost_cname)
            cost_min_idx = cost_total.argmin()

            fit_lat_list.append(candidates_lat[cost_min_idx])
            fit_lng_list.append(candidates_lng[cost_min_idx])
            fit_cname_list.append(candidates_cname[cost_min_idx])
            fit_timediff_list.append(candidates_time[cost_min_idx])
            fit_angle_list.append(None)
            fit_dist_list.append(cost_pred_dist[cost_min_idx])

    df = pd.DataFrame({
        "latDeg": fit_lat_list,
        "lngDeg": fit_lng_list,
        "millisSinceGpsEpoch": query["millisSinceGpsEpoch"],
        "collectionName": query["collectionName"],
        "phoneName": query["phoneName"]
    })
    
    return df

def cost_minimalize(train_df, test_df, gt_df):

    sjc_names = [c for c in collection_names if "SJC" in c]

    sjc_dfs = {
        0: train_df[train_df["collectionName"] == sjc_names[0]].reset_index(drop=True),
        1: train_df[train_df["collectionName"] == sjc_names[1]].reset_index(drop=True),
        2: train_df[train_df["collectionName"] == sjc_names[2]].reset_index(drop=True)
    }
    sjc_gts = {
        0: gt_df[gt_df["collectionName"] == sjc_names[0]].reset_index(drop=True),
        1: gt_df[gt_df["collectionName"] == sjc_names[1]].reset_index(drop=True),
        2: gt_df[gt_df["collectionName"] == sjc_names[2]].reset_index(drop=True)
    }
    sjc_phones = {
        0: sjc_dfs[0]["phoneName"].unique(),
        1: sjc_dfs[1]["phoneName"].unique(),
        2: sjc_dfs[2]["phoneName"].unique()
    }
    train_patterns = [
        [0, [1, 2]],
        [1, [0, 2]],
        [2, [0, 1]],
    ]
    train_fit_df_list = []
    query_df_list = []

    # for train data
    for query_ix, ref_indices in train_patterns:
        print(f"Query {query_ix}")
        for phone_ix in range(len(sjc_phones[query_ix])):

            # queryの指定
            query_all = sjc_dfs[query_ix]

            query = query_all[
                query_all["phoneName"] == sjc_phones[query_ix][phone_ix]
            ].reset_index(drop=True)
            query["speed_pred"] = query["speed_pred"].clip(0, None)

            # referenceの指定
            ref_list = [sjc_gts[ix] for ix in ref_indices]
            ref = pd.concat(ref_list, ignore_index=True)
            
            ret_df = minimalize_search(query, ref)

            train_fit_df_list.append(ret_df)
            query_df_list.append(query)

    train_ret_df = pd.concat(train_fit_df_list, ignore_index=True)
    
    # for test data
    test_fit_df_list = []
    test_ref_indices = [0, 1, 2]
    test_collection_names = test_df["collectionName"].unique()
    
    for c_name in test_collection_names:
        if "SJC" not in c_name:
            continue
        print(f"Test: {c_name}")
        c_test_df = test_df[test_df["collectionName"] == c_name]
        for phone_name in c_test_df["phoneName"].unique():
            
            query = c_test_df[
                c_test_df["phoneName"] == phone_name].reset_index(drop=True)
            query["speed_pred"] = query["speed_pred"].clip(0, None)            
            ref_list = [sjc_gts[ix] for ix in ref_indices]
            ref = pd.concat(ref_list, ignore_index=True)
            
            ret_df = minimalize_search(query, ref)
            test_fit_df_list.append(ret_df)
            
    test_ret_df = pd.concat(test_fit_df_list, ignore_index=True)
    
    return train_ret_df, test_ret_df

In [60]:
target_area = "downtown"
snap_thresh = 100
snap_weight = 1
df_train_area = train_pred[train_pred["area"]==target_area].copy().reset_index(drop=True)
df_test_area = test_pred[test_pred["area"]==target_area].copy().reset_index(drop=True)

In [61]:
train_df, test_df = speed_predict(df_train_area, ground_truth, df_test_area)

<ipython-input-59-c106aa9b72ab>:8: RuntimeWarning: invalid value encountered in double_scalars
  cos = np.inner(vec12, vec32) / (norm12 * norm32)
<ipython-input-59-c106aa9b72ab>:9: RuntimeWarning: invalid value encountered in arccos
  rad = np.arccos(cos)


Fold 0
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[209]	training's rmse: 0.575718	valid_1's rmse: 1.06142
Fold 1
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[88]	training's rmse: 0.805434	valid_1's rmse: 0.811816
Fold 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[397]	training's rmse: 0.505702	valid_1's rmse: 0.998693
RMSE: 0.9688705240859429


In [62]:
collection_names = train_df["collectionName"].unique()

In [63]:
train_ret_df, test_ret_df = cost_minimalize(train_df, test_df, ground_truth)

  0%|          | 0/2889 [00:00<?, ?it/s]

Query 0


  1%|          | 20/2013 [00:00<00:09, 199.59it/s]

Query 1


  1%|          | 21/2329 [00:00<00:11, 203.11it/s]

...

  1%|          | 21/2323 [00:00<00:11, 202.42it/s]

Test: 2021-04-22-US-SJC-2


  1%|          | 21/1978 [00:00<00:09, 203.11it/s]

Test: 2021-04-29-US-SJC-3


100%|██████████| 2060/2060 [00:10<00:00, 202.37it/s]


In [64]:
get_train_score(train_ret_df,ground_truth)

9.62700921517106

In [65]:
train_all.append(train_ret_df)
test_all.append(test_ret_df)

In [66]:
# def rule_for_stg(df,snap_thresh,snap_weight):
#     idx = df["dist_nearest_0"]<snap_thresh
#     df.loc[idx,"latDeg"] = (1-snap_weight)*df.loc[
#         idx,"latDeg"]+snap_weight*df.loc[idx,"latDeg_nearest_0"]
#     df.loc[idx,"lngDeg"] = (1-snap_weight)*df.loc[
#         idx,"lngDeg"]+snap_weight*df.loc[idx,"lngDeg_nearest_0"]
#     return df

In [67]:
# df_train_area = rule_for_stg(df_train_area,snap_thresh,snap_weight)
# df_test_area = rule_for_stg(df_test_area,snap_thresh,snap_weight)

In [68]:
# train_all.append(df_train_area)
# test_all.append(df_test_area)

####  4.4 concat

In [69]:
train_pred = pd.concat(train_all,ignore_index=True)
test_pred = pd.concat(test_all,ignore_index=True)

In [70]:
get_train_score(train_pred, ground_truth,area = True)

area
downtown    9.627009
highway     2.025704
tree        3.117775
Name: p50_p90_mean, dtype: float64


2.859907389338233

In [71]:
train_pred.to_pickle(f"train_ml_{exp_name}.pkl")
test_pred.to_pickle(f"test_ml_{exp_name}.pkl")

In [72]:
train_pred =pd.read_pickle(f"train_ml_{exp_name}.pkl")
test_pred =pd.read_pickle(f"test_ml_{exp_name}.pkl")

In [73]:
train_pred["phone"] = train_pred["collectionName"]+"_"+train_pred["phoneName"]
test_pred["phone"] = test_pred["collectionName"]+"_"+test_pred["phoneName"]
train_pred["area"] = train_pred["collectionName"].map(collections_to_areas)
test_pred["area"] = test_pred["collectionName"].map(collections_to_areas)

In [74]:
get_train_score(train_pred, ground_truth,area = True)

area
downtown    9.627009
highway     2.025704
tree        3.117775
Name: p50_p90_mean, dtype: float64


2.859907389338233

In [75]:
train_pred_all = []
test_pred_all = []
#### 2.1 highway
target_area = 'highway'
thresh_stopping = 2
median_window = 30
repeats = 1
phone_weights = [
    0.8185272369412809,
    0.34843268384881615,
    0.4846644591847249,
    0.9452238465987974,
    0.9986981935560018,
    0.18031216717585496,
    0.445328696935905
    ]

train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = repeat_postprocess(
    train_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    evalate=True
)
train_pred_all.append(train_pred_area)

count : 1, score : 2.0147


In [76]:

test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = repeat_postprocess(
    test_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
test_pred_all.append(test_pred_area)

In [77]:


####  2.2tree

target_area = 'tree'
thresh_stopping = 1.5
median_window = 30
repeats = 6
phone_weights = [
    0.3115193764226983,
    0.196919988137904,
    0.1757293314492492,
    0.4811756152653266,
    0.9662722758231396,
    0.48662174873037445,
    0.1015768847715516
]

train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = repeat_postprocess(
    train_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    evalate=True
)
train_pred_all.append(train_pred_area)

test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = repeat_postprocess(
    test_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
test_pred_all.append(test_pred_area)

count : 1, score : 3.1163
count : 2, score : 3.1165
count : 3, score : 3.1123
count : 4, score : 3.1117
count : 5, score : 3.1120
count : 6, score : 3.1083


In [78]:
#### 2.3 downtown

target_area = 'downtown'
thresh_stopping = 2
median_window = 17
repeats = 1
phone_weights = [
    0.30701282127989166,
    0.8541717731249667,
    0.1375108003326193,
    0.34089358646969836,
    0.3585169573830474,
    0.253223739710727,
    0.4830857599143877
]

train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = repeat_postprocess(
    train_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    evalate=True
)
train_pred_all.append(train_pred_area)


count : 1, score : 9.4646


In [79]:

test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = repeat_postprocess(
    test_pred_area,
    thresh_stopping,
    median_window,
    repeats,
    phone_weights,
    #evalate=True
)
test_pred_all.append(test_pred_area)

train_pred = pd.concat(train_pred_all,ignore_index=True)
test_pred = pd.concat(test_pred_all,ignore_index=True)

In [80]:
get_train_score(train_pred,ground_truth,area=True)

area
downtown    9.464606
highway     2.014750
tree        3.108288
Name: p50_p90_mean, dtype: float64


2.836786462394463

In [81]:
train_pred["phone"] = train_pred["collectionName"]+"_"+train_pred["phoneName"]
test_pred["phone"] = test_pred["collectionName"]+"_"+test_pred["phoneName"]
train_pred["area"] = train_pred["collectionName"].map(collections_to_areas)
test_pred["area"] = test_pred["collectionName"].map(collections_to_areas)

In [82]:
import pyproj
from pyproj import Proj, transform

In [83]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z

transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)
def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt

In [84]:
def position_shift(d,a):
    cols = d.columns
    d['heightAboveWgs84EllipsoidM'] = 63.5
    d['x'], d['y'], d['z'] = zip(*d.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, x.heightAboveWgs84EllipsoidM), axis=1))

    #a = -0.2
    d.sort_values(['phone','millisSinceGpsEpoch'], inplace=True)
    for fi in ['x','y','z']:
        d[[fi+'p']] = d[fi].shift().where(d['phone'].eq(d['phone'].shift()))
        d[[fi+'diff']] = d[fi]-d[fi+'p']
    #d[['yp']] = d['y'].shift().where(d['phone'].eq(d['phone'].shift()))
    d[['dist']] = np.sqrt(d['xdiff']**2 + d['ydiff']**2+ d['zdiff']**2)
    for fi in ['x','y','z']:
        d[[fi+'new']] = d[fi+'p'] + d[fi+'diff']*(1-a/d['dist'])
    lng, lat, alt = ECEF_to_WGS84(d['xnew'].values,d['ynew'].values,d['znew'].values)
    lng[np.isnan(lng)] = d.loc[np.isnan(lng),'lngDeg']
    lat[np.isnan(lat)] = d.loc[np.isnan(lat),'latDeg']
    d['latDeg'] = lat
    d['lngDeg'] = lng

    d.sort_values(['phone','millisSinceGpsEpoch'],inplace = True)    
    return d[cols]

In [85]:
train_all = []
test_all = []

In [86]:
a = 0.15
target_area = "highway"

In [87]:
train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = position_shift(train_pred_area,a)
train_all.append(train_pred_area)
test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = position_shift(test_pred_area,a)
test_all.append(test_pred_area)

In [88]:
a = 0.4
target_area = "tree"

In [89]:
train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = position_shift(train_pred_area,a)
train_all.append(train_pred_area)
test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = position_shift(test_pred_area,a)
test_all.append(test_pred_area)

In [90]:
a = 1.35
target_area = "downtown"

In [91]:
train_pred_area = train_pred[
    train_pred["area"]==target_area].reset_index(drop=True)
train_pred_area = position_shift(train_pred_area,a)
train_all.append(train_pred_area)
test_pred_area = test_pred[
    test_pred["area"]==target_area].reset_index(drop=True)
test_pred_area = position_shift(test_pred_area,a)
test_all.append(test_pred_area)

In [92]:
train_pred = pd.concat(train_all,ignore_index=True)
test_pred = pd.concat(test_all,ignore_index=True)

In [93]:
get_train_score(train_pred,ground_truth,area = True)

area
downtown    9.238825
highway     1.996353
tree        3.024461
Name: p50_p90_mean, dtype: float64


2.788796419204225

In [94]:
#visualize_collection_with_gt(train_pred, ground_truth,'2020-05-21-US-MTV-1')

In [95]:
#visualize_collection_with_gt(train_pred, ground_truth,'2020-09-04-US-SF-1')

### 5. submit

In [96]:
test_pred["phone"] = test_pred["collectionName"]+ '_' + test_pred['phoneName']

In [97]:
sub = sample_sub[["phone","millisSinceGpsEpoch"]].copy()
sub = sub.merge(test_pred[[
    "phone","millisSinceGpsEpoch","latDeg","lngDeg"]],
                on=["phone","millisSinceGpsEpoch"],how="left")

sub.to_csv(f'sub_{exp_name}.csv', index=False)